In [1]:
!ls -alh

total 491M
drwxrwxr-x  5 hoang hoang 4,0K Thg 11 15 20:00 .
drwxrwxr-x 12 hoang hoang 4,0K Thg 11 14 00:36 ..
drwxr-xr-x  4 hoang hoang 4,0K Thg 9   5 10:51 DATA_CHAMBER_2021
-rw-rw-r--  1 hoang hoang 491M Thg 11 14 00:36 DATA_CHAMBER_2021.zip
drwxrwxr-x  2 hoang hoang 4,0K Thg 11 14 16:17 .ipynb_checkpoints
drwxrwxr-x  3 hoang hoang 4,0K Thg 11 14 01:02 __MACOSX
-rw-rw-r--  1 hoang hoang 9,4K Thg 11 15 20:00 Untitled1.ipynb
-rw-rw-r--  1 hoang hoang 5,6K Thg 11 15 04:31 Untitled2.ipynb
-rw-rw-r--  1 hoang hoang  70K Thg 11 14 01:23 Untitled.ipynb


In [2]:
train_dataset_path = './DATA_CHAMBER_2021/train'
test_dataset_path = './DATA_CHAMBER_2021/test'

In [3]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [4]:
os.listdir(train_dataset_path)

['4C', '2C', '3C', '.DS_Store']

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
train_datasets = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = transform)
test_datasets = torchvision.datasets.ImageFolder(root = test_dataset_path, transform = transform)

In [7]:
train_loader = DataLoader(train_datasets, batch_size = 8, shuffle = True, num_workers = 2)
test_loader = DataLoader(test_datasets, batch_size = 8, shuffle = False, num_workers = 2)

# THIẾT BỊ

In [8]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)
set_device()

device(type='cpu')

In [9]:
def train_model(model, train_loader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch+1, n_epochs))
        print('-' * 10)
        
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        print('{} loss: {:.4f}, acc: {:.4f}'.format(total, epoch_loss, epoch_acc))

    return model

# RESNET18

In [10]:
resnet18_model = models.resnet18(pretrained=True)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 3
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet_18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/hoang/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
train_model(resnet18_model, train_loader, loss_fn, optimizer, 20)

Epoch 1/20
----------
6717 loss: 0.4822, acc: 87.7028
Epoch 2/20
----------
6717 loss: 0.0367, acc: 98.9132
Epoch 3/20
----------
6717 loss: 0.0364, acc: 98.8983
Epoch 4/20
----------
6717 loss: 0.0242, acc: 99.3449
Epoch 5/20
----------
6717 loss: 0.0303, acc: 99.0919
Epoch 6/20
----------
6717 loss: 0.0234, acc: 99.4194
Epoch 7/20
----------
6717 loss: 0.0407, acc: 99.0174
Epoch 8/20
----------
6717 loss: 0.0364, acc: 99.1514
Epoch 9/20
----------
6717 loss: 0.0315, acc: 99.2407
Epoch 10/20
----------
6717 loss: 0.0252, acc: 99.4938
Epoch 11/20
----------
6717 loss: 0.0083, acc: 100.0000
Epoch 12/20
----------
6717 loss: 0.0829, acc: 97.2756
Epoch 13/20
----------
6717 loss: 0.0310, acc: 99.3747
Epoch 14/20
----------
6717 loss: 0.0381, acc: 99.0621
Epoch 15/20
----------
6717 loss: 0.0344, acc: 99.2556
Epoch 16/20
----------
6717 loss: 0.0422, acc: 99.1216
Epoch 17/20
----------
6717 loss: 0.0345, acc: 99.1812
Epoch 18/20
----------
6717 loss: 0.0222, acc: 99.5385
Epoch 19/20
------

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
def test_model(model, test_loader, criterion, optimizer):
    labels_input=list()
    labels_output=list()
    vid_id = list()
        
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels_input= labels_input + labels.tolist()
    
        outputs = model(images)
            
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
            
        labels_output= labels_output + preds.tolist()
    return labels_input,labels_output

y_true,y_pred = test_model(resnet18_model, test_loader, loss_fn, optimizer)

In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(classification_report(y_true,y_pred))
accuracy_score(y_true, y_pred)

              precision    recall  f1-score   support

           0       0.86      1.00      0.93       409
           1       0.99      0.88      0.94       367
           2       1.00      0.97      0.99       831

    accuracy                           0.96      1607
   macro avg       0.95      0.95      0.95      1607
weighted avg       0.96      0.96      0.96      1607



0.9583074051026758